In [ ]:
import os
import json
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install dgl dglgo -f https://data.dgl.ai/wheels/repo.html

In [ ]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from dgl import DGLGraph
import re
import scipy.sparse as sp

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
ratings = []
with open(os.path.join('/content/gdrive/MyDrive/Colab Notebooks/인턴/ml-1m/', 'ratings.dat'), encoding='latin1') as f:
    for l in f:
        user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
        ratings.append({
            'user_id': user_id,
            'movie_id': movie_id,
            'rating': rating,
            'timestamp': timestamp,
        })
ratings = pd.DataFrame(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
len(ratings.index)

1000209

In [ ]:
len(ratings['user_id'])

1000209

In [ ]:
len(ratings['user_id'].unique()) 

6040

In [ ]:
max(list(ratings['user_id'].unique()))

6040

In [ ]:
user_uni = ratings['user_id'].unique() # 유저 id는 1부터 6040까지 연속된 숫자 
for i in range(1, 6041):
  if i not in user_uni:
    print('d')

In [ ]:
min(ratings['movie_id'].values) #movieid의 최소값이 1이니 6040을 더하면 겹치지 않을 것 

1

In [ ]:
np_movieid = ratings['movie_id'].values
np_movieid

array([1193,  661,  914, ...,  562, 1096, 1097])

In [ ]:
npplus = np.full((1000209,), 6040)
npplus

array([6040, 6040, 6040, ..., 6040, 6040, 6040])

In [ ]:
ratings['movie_id_plus'] = list(np_movieid +  npplus)   #ratings['movie_id_plus']는 영화 id + 6040
ratings.head()

,user_id,movie_id,rating,timestamp,movie_id_plus
0,1,1193,5,978300760,7233
1,1,661,3,978302109,6701
2,1,914,3,978301968,6954
3,1,3408,4,978300275,9448
4,1,2355,5,978824291,8395


In [ ]:
user_id = ratings['user_id'].unique()
movie_id = ratings['movie_id'].unique()

In [ ]:
user_id_len = len(user_id)
movie_id_len = len(movie_id)
totalid = user_id_len + movie_id_len                   #total id 개수는 9746 
totalid

9746

In [ ]:
user_id_len, movie_id_len                             # 노드 개수 => 유저: 6040 영화 3706 

(6040, 3706)

In [ ]:
movie_id = np.sort(movie_id)
movie_id

array([   1,    2,    3, ..., 3950, 3951, 3952])

In [ ]:
nodeidx = {}   #key는 user_id, movie_id/ value는 node id 
id = 0         #노드 id는 0부터 9745 (마지막에 id는 테이블에 넣지 않았으니)

for i in range(user_id_len):
  nodeidx[user_id[i]] = id
  id += 1
print(id)     #node id 0~6039는 유저거 

6040


In [ ]:
movie_id[0] + 6040

6041

In [ ]:
for j in range(movie_id_len): # 노드 id 6040 ~ 9745는 무비거 
  nodeidx[movie_id[j] + 6040]  = id
  id += 1
print(id)

9746


In [ ]:
nodeidx[1]

0

In [ ]:
nodeidx[6040]

6039

In [ ]:
nodeidx[6041] 

6040

In [ ]:
nodeidx[9992] #movie id는 제일 큰 거 

9745

In [ ]:
len(nodeidx)

9746

In [ ]:
ratings.head(10)

,user_id,movie_id,rating,timestamp,movie_id_plus
0,1,1193,5,978300760,7233
1,1,661,3,978302109,6701
2,1,914,3,978301968,6954
3,1,3408,4,978300275,9448
4,1,2355,5,978824291,8395
5,1,1197,3,978302268,7237
6,1,1287,5,978302039,7327
7,1,2804,5,978300719,8844
8,1,594,4,978302268,6634
9,1,919,4,978301368,6959


In [ ]:
 ex = ratings.iloc[0, 4]
 ex

7233

In [ ]:
users = []  #user_id에 해당하는 노드 id를 넣음 
movies = []

for i in range(len(ratings.index)):
  user = ratings.iloc[i,0]
  users.append(nodeidx[user])

  movie = ratings.iloc[i, 4]
  movies.append(nodeidx[movie])


In [ ]:
len(users), len(movies)

(1000209, 1000209)

In [ ]:
edges_src = torch.from_numpy(np.array(users))
edges_dst = torch.from_numpy(np.array(movies))

In [ ]:
graph = dgl.graph((edges_src, edges_dst))
print(graph)

Graph(num_nodes=9746, num_edges=1000209,
      ndata_schemes={}
      edata_schemes={})


In [ ]:
edge_features = torch.from_numpy(ratings['rating'].to_numpy())
len(edge_features)

1000209

In [ ]:
graph.edata['w'] = edge_features

In [ ]:
print(graph)

Graph(num_nodes=9746, num_edges=1000209,
      ndata_schemes={}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.int64)})


https://docs.dgl.ai/tutorials/blitz/4_link_predict.html?highlight=negative

In [ ]:
u, v = graph.edges() #엣지 길이 

In [ ]:
eids = np.arange(graph.number_of_edges()) #엣지를 가지고 train, test로 나누어줌

eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)    #엣지 중 10%를 택함
train_size = graph.number_of_edges() - test_size

test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

In [ ]:
#여기서부터 node pairs with no edges between them as negative examples.
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())), shape=(9746,9746)      )
print(adj.shape)
adj_neg = 1 - adj.todense() - np.eye(graph.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

(9746, 9746)


In [ ]:
neg_eids = np.random.choice(len(neg_u), graph.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [ ]:
train_g = dgl.remove_edges(graph, eids[:test_size])

In [ ]:

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=graph.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=graph.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=graph.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=graph.number_of_nodes())


In [ ]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
    
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [ ]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [ ]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.
        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.
        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

In [ ]:
model = GraphSAGE(9746, 16)
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [ ]:
import itertools
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

In [ ]:
feat = torch.zeros(9746, 9746) + torch.eye(9746)
feat

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [ ]:
feat.shape

torch.Size([9746, 9746])

In [ ]:
all_logits = []
for e in range(100):
    # forward
    h = model(train_g, feat)
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

In epoch 0, loss: 0.6932427287101746
In epoch 5, loss: 0.6619951725006104
In epoch 10, loss: 0.5943281650543213
In epoch 15, loss: 0.5724420547485352
In epoch 20, loss: 0.5647295117378235
In epoch 25, loss: 0.5594874620437622
In epoch 30, loss: 0.5558831095695496
In epoch 35, loss: 0.5498736500740051
In epoch 40, loss: 0.5414936542510986
In epoch 45, loss: 0.5335385203361511
In epoch 50, loss: 0.5265455842018127
In epoch 55, loss: 0.5219001770019531
In epoch 60, loss: 0.5189764499664307
In epoch 65, loss: 0.5170321464538574
In epoch 70, loss: 0.5154984593391418
In epoch 75, loss: 0.5141327977180481
In epoch 80, loss: 0.512814462184906
In epoch 85, loss: 0.5115363597869873
In epoch 90, loss: 0.5102699995040894
In epoch 95, loss: 0.5088672637939453


In [ ]:
h.shape

torch.Size([9746, 16])

In [ ]:
# 벡터 유사도 측정 함수
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [ ]:
nodeidx[9742], nodeidx[9743], nodeidx[9744], nodeidx[7648], nodeidx[7761], nodeidx[6151], nodeidx[6255]

(9500, 9501, 9502, 7516, 7614, 6147, 6249)

In [ ]:
#에어포스원: 764, 타이타닉: 7761, 택시드라이버: 6151, 비포선라이즈: 6255

In [ ]:
cos_sim(h[9501].detach().numpy(),h[9500].detach().numpy() )

0.98707634

In [ ]:
cos_sim(h[9501].detach().numpy(),h[9502].detach().numpy() )

0.8221346

In [ ]:
cos_sim(h[9501].detach().numpy(), h[7516].detach().numpy() )

0.85226387

In [ ]:
cos_sim(h[9501].detach().numpy(),h[7614].detach().numpy() )

0.97317624

In [ ]:
cos_sim(h[9501].detach().numpy(), h[6147].detach().numpy() )

0.83457416

In [ ]:
cos_sim(h[9501].detach().numpy(),h[6249].detach().numpy() )

-0.3955274

In [ ]:
nodeidx[7890] #예전에 선택했던 무비노드 

7710

In [ ]:
cos_sim(h[7710].detach().numpy(), h[9].detach().numpy() )   #예전에 선택했던만큼 나온다. 

-0.9019539

In [ ]:
cos_sim(h[7710].detach().numpy(), h[0].detach().numpy() )

-0.30725974

In [ ]:
cos_sim(h[6041].detach().numpy(), h[0].detach().numpy() )

0.16116253